In [1]:
%matplotlib qt

import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

# Parte 1

## 1.1 Coordenadas cromáticas

In [36]:
img1 = cv.imread('data/coord_cromaticas/CoordCrom_1.png')
img2 = cv.imread('data/coord_cromaticas/CoordCrom_2.png')
img3 = cv.imread('data/coord_cromaticas/CoordCrom_3.png')

In [109]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))  # 1 fila, 3 columnas
ax1.imshow(img1)
ax1.set_title('Imagen 1')
ax2.imshow(img2)
ax2.set_title('Imagen 2')
ax3.imshow(img3)
ax3.set_title('Imagen 3')
plt.show()

In [35]:
img1.shape

(996, 908, 3)

In [89]:
def convert_to_cromatic_coords(img: np.ndarray) -> np.ndarray:
    '''
    Parameters
    ----------
    img : np.ndarray
        Input image in BGR format.

    Returns
    -------
    np.ndarray
        Image converted to chromatic coordinates.
    '''
    img_float = img.astype(np.float32)  # Convertir a float32 para precisión en la división
    img_cromatic = np.zeros_like(img_float)

    for row in range(img.shape[0]):
        for col in range(img.shape[1]):
            b, g, r = img_float[row, col]
            sum_channels = b + g + r
            if sum_channels > 0:
                img_cromatic[row, col] = [b / sum_channels, g / sum_channels, r / sum_channels]

    # Convertir de nuevo a 0-255 y uint8 para visualización
    img_cromatic = (img_cromatic * 255).astype(np.uint8)

    return img_cromatic

### Resultados

In [111]:
img1_cromatic = convert_to_cromatic_coords(img1)
img2_cromatic = convert_to_cromatic_coords(img2)
img3_cromatic = convert_to_cromatic_coords(img3)


fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, figsize=(15, 10))

ax1.imshow(cv.cvtColor(img1, cv.COLOR_BGR2RGB))
ax1.set_title('Imagen 1 - Original')

ax2.imshow(cv.cvtColor(img2, cv.COLOR_BGR2RGB))
ax2.set_title('Imagen 2 - Original')

ax3.imshow(cv.cvtColor(img3, cv.COLOR_BGR2RGB))
ax3.set_title('Imagen 3 - Original')

ax4.imshow(cv.cvtColor(img1_cromatic, cv.COLOR_BGR2RGB))
ax4.set_title('Imagen 1 - Coordenadas Cromáticas')

ax5.imshow(cv.cvtColor(img2_cromatic, cv.COLOR_BGR2RGB))
ax5.set_title('Imagen 2 - Coordenadas Cromáticas')

ax6.imshow(cv.cvtColor(img3_cromatic, cv.COLOR_BGR2RGB))
ax6.set_title('Imagen 3 - Coordenadas Cromáticas')

plt.tight_layout()
plt.show()

## 1.2 White patch

In [37]:
test_blue = cv.imread('data/white_patch/test_blue.png')
test_green = cv.imread('data/white_patch/test_green.png')
test_red = cv.imread('data/white_patch/test_red.png')

In [46]:
wp_blue = cv.imread('data/white_patch/wp_blue.jpg')
wp_green = cv.imread('data/white_patch/wp_green.png')
wp_green2 = cv.imread('data/white_patch/wp_green2.jpg')
wp_red = cv.imread('data/white_patch/wp_red.png')
wp_red2 = cv.imread('data/white_patch/wp_red2.jpg')

In [111]:
def white_patch(img: np.ndarray) -> np.ndarray:
    '''
    Parameters
    ----------
    img : np.ndarray
        Input image in BGR format.

    Returns
    -------
    np.ndarray
        Image patched in BGR format.
    '''
    img_float = img.astype(np.float32)  # Convertir a float32 para precisión en la división
    adjust_max = lambda channel: np.percentile(channel, 95) if channel.max() == 255 else channel.max()
    #adjust_max = lambda channel: np.percentile(channel, 95) if channel.max() == 255 else channel.max()-50 # testing

    # máximo ajustado de cada canal
    b_max = adjust_max(img_float[:,:,0])
    g_max = adjust_max(img_float[:,:,1])
    r_max = adjust_max(img_float[:,:,2])
    
    # Verificar si alguno de los canales tiene un valor máximo de 0
    if b_max == 0 or g_max == 0 or r_max == 0:
        raise ValueError("Problemas con la cámara: uno de los canales tiene un valor máximo de 0.")

    # Normalizar cada canal por su valor máximo
    img_patched = img_float / [b_max, g_max, r_max] * 255.0

    # Convertir de nuevo a uint8 para visualización
    img_patched = img_patched.astype(np.uint8)

    return img_patched

### Resultados

In [112]:
# Crear figura y subplots
fig, axs = plt.subplots(2, 3, figsize=(15, 8))

# Mostrar imágenes originales en la primera fila
axs[0, 0].imshow(cv.cvtColor(test_blue, cv.COLOR_BGR2RGB))
axs[0, 0].set_title('Original Test Blue')
axs[0, 1].imshow(cv.cvtColor(test_green, cv.COLOR_BGR2RGB))
axs[0, 1].set_title('Original Test Green')
axs[0, 2].imshow(cv.cvtColor(test_red, cv.COLOR_BGR2RGB))
axs[0, 2].set_title('Original Test Red')

# Mostrar imágenes parcheadas en la segunda fila
axs[1, 0].imshow(cv.cvtColor(white_patch(test_blue), cv.COLOR_BGR2RGB))
axs[1, 0].set_title('Patched Test Blue')
axs[1, 1].imshow(cv.cvtColor(white_patch(test_green), cv.COLOR_BGR2RGB))
axs[1, 1].set_title('Patched Test Green')
axs[1, 2].imshow(cv.cvtColor(white_patch(test_red), cv.COLOR_BGR2RGB))
axs[1, 2].set_title('Patched Test Red')

# Ajustar espacios y mostrar la figura
plt.tight_layout()
plt.show()

In [118]:
# Crear figura y subplots
fig, axs = plt.subplots(2, 5, figsize=(15, 8))
# Mostrar imágenes originales en la primera fila
axs[0, 0].imshow(cv.cvtColor(wp_blue, cv.COLOR_BGR2RGB))
axs[0, 0].set_title('Original WP Blue')
axs[0, 1].imshow(cv.cvtColor(wp_green, cv.COLOR_BGR2RGB))
axs[0, 1].set_title('Original WP Green')
axs[0, 2].imshow(cv.cvtColor(wp_green2, cv.COLOR_BGR2RGB))
axs[0, 2].set_title('Original WP Green 2')
axs[0, 3].imshow(cv.cvtColor(wp_red, cv.COLOR_BGR2RGB))
axs[0, 3].set_title('Original WP Red')
axs[0, 4].imshow(cv.cvtColor(wp_red2, cv.COLOR_BGR2RGB))
axs[0, 4].set_title('Original WP Red 2')
# Mostrar imágenes parcheadas en la segunda fila
axs[1, 0].imshow(cv.cvtColor(white_patch(wp_blue), cv.COLOR_BGR2RGB))
axs[1, 0].set_title('Patched WP Blue')
axs[1, 1].imshow(cv.cvtColor(white_patch(wp_green), cv.COLOR_BGR2RGB))
axs[1, 1].set_title('Patched WP Green')
axs[1, 2].imshow(cv.cvtColor(white_patch(wp_green2), cv.COLOR_BGR2RGB))
axs[1, 2].set_title('Patched WP Green 2')
axs[1, 3].imshow(cv.cvtColor(white_patch(wp_red), cv.COLOR_BGR2RGB))
axs[1, 3].set_title('Patched WP Red')
axs[1, 4].imshow(cv.cvtColor(white_patch(wp_red2), cv.COLOR_BGR2RGB))
axs[1, 4].set_title('Patched WP Red 2')
# Ajustar espacios y mostrar la figura
plt.tight_layout()
plt.show()

# 1.3 Falla Whitepatch

Se observa una "falla" cuando se trabaja con la imagen wp_blue. Esto tiene una explicación y una posible mitigación.

- Explicación: Viendo la distribución de intensidades en los canales de la imagen wp_blue, vemos en la imagen que genera la celda posterior que el percentil seleccionado para los canales R y G responden a valores muy bajos. Por ende el escalamiento afecta en gran medida a los valores "bajos" que poseían de antemano estos canales.

- Mitigación: Tomar un percentil superior para evitar una disminución de escala abrupta por la baja densidad de intensidades altas que presentan las distribuciones de algunos canales. Por ejemplo: Usar el percentil 99.
Esto mitiga el problema en la imagen wp_blue pero baja el rendimiento general del algoritmo, que se puede observar en la modificación de rendimiento que recibe el resto de las imágenes. 

In [117]:
# Leer la imagen
wp_blue = cv.imread('data/white_patch/wp_blue.jpg')

# Convertir de BGR a RGB para mostrar con Matplotlib
wp_blue_rgb = cv.cvtColor(wp_blue, cv.COLOR_BGR2RGB)

# Calcular histogramas para cada canal de color
hist_b = cv.calcHist([wp_blue], [0], None, [256], [0, 256])
hist_g = cv.calcHist([wp_blue], [1], None, [256], [0, 256])
hist_r = cv.calcHist([wp_blue], [2], None, [256], [0, 256])

# Calcular percentil 95 para cada canal
percentile_b = np.percentile(wp_blue[:,:,0], 95)
percentile_g = np.percentile(wp_blue[:,:,1], 95)
percentile_r = np.percentile(wp_blue[:,:,2], 95)

# Crear la figura y los subplots
fig, axs = plt.subplots(1, 4, figsize=(15, 4))

# Mostrar la imagen
axs[0].imshow(wp_blue_rgb)
axs[0].set_title('Imagen')

# Mostrar histograma para el canal Azul (B)
axs[1].plot(hist_b, color='b')
axs[1].axvline(x=percentile_b, color='r', linestyle='--', label=f'Percentil 95 ({int(percentile_b)})')
axs[1].set_title('Histograma Azul')
axs[1].set_xlim([0, 256])
axs[1].legend()

# Mostrar histograma para el canal Verde (G)
axs[2].plot(hist_g, color='g')
axs[2].axvline(x=percentile_g, color='r', linestyle='--', label=f'Percentil 95 ({int(percentile_g)})')
axs[2].set_title('Histograma Verde')
axs[2].set_xlim([0, 256])
axs[2].legend()

# Mostrar histograma para el canal Rojo (R)
axs[3].plot(hist_r, color='r')
axs[3].axvline(x=percentile_r, color='r', linestyle='--', label=f'Percentil 95 ({int(percentile_r)})')
axs[3].set_title('Histograma Rojo')
axs[3].set_xlim([0, 256])
axs[3].legend()

# Ajustar espacios y mostrar la figura
plt.tight_layout()
plt.show()


(TO DO) agregar nuevo codigo para display de la imagen wp_blue con los g_max y r_max hardcodeados a 240 aprox.

---

# Parte 2

## 2.1 Escala de grises